In [34]:
import time,os,requests,io
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from scipy import optimize

### Read Input & add Nifty Data

In [35]:
# Read the Transactions File
tdf = pd.read_csv("npsTransactions.csv")
def negUnits(x):
    try:
        x = float(x)
    except:
        if(x[0] == "("):
            x = -float(x[1:-1])
        else:
            print("error")
    return x
tdf['Amt'] = tdf['Amt'].apply(negUnits)

tdf['Date'] = pd.to_datetime(tdf['Date'])
tdf = tdf.sort_values(by='Date',ascending=True, ignore_index=True)
tdf['Date'] = tdf['Date'].dt.strftime('%Y-%m-%d')

fdate = datetime.strptime(min(tdf["Date"]), "%Y-%m-%d") - timedelta(days = 2)
tdate = datetime.today()

In [36]:
def getNifty(date):
	url="https://archives.nseindia.com/content/indices/ind_close_all_%s.csv"%date
	s=requests.get(url).content
	indicesDf=pd.read_csv(io.StringIO(s.decode('utf-8')))
	indicesDf = indicesDf.loc[indicesDf['Index Name'] == 'Nifty 50']
	niftyClose = indicesDf.at[0,'Closing Index Value']
	return float(niftyClose/100)

df = tdf
for i in range(len(df['Date'])):
    inv = float(df.at[i,'Amt'])
    date = datetime.strptime(df.at[i,'Date'], "%Y-%m-%d")
    try:
        if(str(df.at[i,"Nifty NAV"]) == "nan"): 
            print("Nifty",date)
            df.at[i,"Nifty NAV"] = getNifty(date.strftime("%d%m%Y"))
    except:
        date = date - timedelta(days=1)
        print("Nifty-1",date)
        df.at[i,"Nifty NAV"] = getNifty(date.strftime("%d%m%Y"))

df = pd.read_csv("npsTransactions.csv")
df["Nifty NAV"] = tdf["Nifty NAV"]
#df.to_csv("npsTransactions.csv",index=False)

Nifty-1 2022-04-08 00:00:00
Nifty 2022-04-20 00:00:00
Nifty 2022-06-13 00:00:00
Nifty 2022-07-09 00:00:00
Nifty-1 2022-07-08 00:00:00
Nifty 2022-07-18 00:00:00
Nifty 2022-08-19 00:00:00
Nifty 2022-09-01 00:00:00
Nifty 2023-03-13 00:00:00
Nifty 2023-04-06 00:00:00
Nifty 2023-04-11 00:00:00
Nifty 2023-05-08 00:00:00
Nifty 2023-06-12 00:00:00


### Get PFM NAV

In [37]:
PFMs = {"SBI":"001", "HDFC":"008", "ICICI":"007"}
tiers = {"E1":1, "E2":4}
offsets = {"SBI":2, "HDFC":0, "ICICI":0}

In [38]:
dfNAV = pd.DataFrame()
for pfm in PFMs.keys():
    for tier in tiers.keys():
        o = requests.get(f"https://www.npstrust.org.in/nav-graphs-details?lnavdata=PFM{PFMs[pfm]}&yearval={fdate.strftime('%Y-%m-%d')}@{tdate.strftime('%Y-%m-%d')}&subcat=SM{PFMs[pfm]}00{offsets[pfm]+tiers[tier]}")

        date = []
        nav = []
        for d in o.text[2:-2].split("],["):
            d = d.split(",")
            date.append(datetime.fromtimestamp(int(d[0][:-3])))
            nav.append(d[1])

        dfNAV["Date"] = pd.to_datetime(date).strftime("%Y-%m-%d")
        dfNAV[f"{pfm}_{tier}"] = nav

#dfNAV.to_csv(f"NPS_E_NAV_Data.csv",index=False)

### Compute

In [39]:
# Initialization and functions
def pfmNAV(date,pfm,tier="E1"):
    return float(dfNAV.loc[(dfNAV["Date"]) == "%s"%date][f"{pfm}_{tier}"]) 

# Functions
def xnpv(rate,cashflows):
    chron_order = sorted(cashflows, key = lambda x: x[0])
    t0 = chron_order[0][0]
    return sum([float(cf)/(1+rate)**((t-t0).days/365.0) for (t,cf) in chron_order])

def xirr(cashflows,currDate,current,guess=0.1):
	for i in range(len(cashflows)):
		cashflows[i][0] = datetime.strptime(cashflows[i][0], "%Y-%m-%d") 
	cashflows.append((currDate,-current))
	return optimize.newton(lambda r: xnpv(r,cashflows),guess)

In [40]:
# Units alloted if invested in SBI vs HDFC vs Nifty
df = tdf.copy()
# Nifty
pfm = "Nifty"
df[f"{pfm} Units"] = df["Amt"]/df[f"{pfm} NAV"]
df[f"{pfm} Units"] = df[f"{pfm} Units"].apply(lambda x: round(x, 4))
df = df.drop(columns=[f"{pfm} NAV", 'NAV'])

for pfm in PFMs.keys():
    df[f"{pfm} Units"] = np.zeros(len(df['Date']))

    for i in range(len(df['Date'])):
        inv = float(df.at[i,'Amt'])
        date = datetime.strptime(df.at[i,'Date'], "%Y-%m-%d")

        try:
            df.at[i,f"{pfm} Units"] = inv/pfmNAV(date.strftime("%Y-%m-%d"),pfm)
        except:
            date = date - timedelta(days=1)
            df.at[i,f"{pfm} Units"] = inv/pfmNAV(date.strftime("%Y-%m-%d"),pfm)

    df[f"{pfm} Units"] = df[f"{pfm} Units"].apply(lambda x: round(x, 4))


index = len(df['Date'])
dateT = datetime.strptime(list(dfNAV["Date"])[-1], "%Y-%m-%d")

pfm = "Nifty"
Curr = (df[f'{pfm} Units'].sum())*getNifty(dateT.strftime("%d%m%Y"))
xirr1 = 100*xirr(((df[["Date", "Amt"]]).values.tolist()),dateT,Curr)
df.at[index,f"{pfm} Units"] = round(Curr,2)
df.at[index+1,f"{pfm} Units"] = f"{round(xirr1,2)}%"

for pfm in PFMs.keys():
    Curr = (df[f'{pfm} Units'].sum())*pfmNAV(dateT.strftime("%Y-%m-%d"),pfm)
    xirr1 = 100*xirr(((df[["Date", "Amt"]]).values.tolist())[:-2],dateT,Curr)
    df.at[index,f"{pfm} Units"] = round(Curr,2)   
    df.at[index+1,f"{pfm} Units"] = f"{round(xirr1,2)}%"

df.at[index,"Date"] = dateT.strftime("%d-%b-%Y")
df.at[index,"Amt"] = df['Amt'].sum()

In [41]:
# Save to file
df.to_csv("NPS_Tier1Equity_Analysis.csv",index=False)